In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from nlp.models import Tfidf, TopicModelSVD
from nlp.util import *
from wrangling.cast import CastingPool

from pymongo import MongoClient
from sklearn.cluster import KMeans


cpool = CastingPool()


## Grab two events
event_a = cpool.db.events.find_one({'title': 'Man Lands on the Moon'})
event_b = cpool.db.events.find_one({'title': 'OJ Simpson Trial'})

In [2]:
## Grab two sets of roles
a_roles = cpool.get_roles(event_a['_id'])
b_roles = cpool.get_roles(event_b['_id'])

In [3]:
## Grab two casting pools
#a_cstng_pool = cpool.get_potential_cast(event_a['_id'])
#b_cstng_pool = cpool.get_potential_cast(event_b['_id'])

#Casting Pool Will Be The Same
casting_pool = cpool.get_casting_pool()

In [4]:
## Topic Model Cast A, Cast B

#docs_a = [p['stemmed_wiki_doc'] for p in a_cstng_pool]
#doc_ids_a = [p['_id'] for p in a_cstng_pool]

#model_a = Tfidf(docs_a, doc_ids_a)
#topic_model_a = TopicModelSVD(model_a.model, model_a.cv_tfidf)


#docs_b = [p['stemmed_wiki_doc'] for p in b_cstng_pool]
#doc_ids_b = [p['_id'] for p in b_cstng_pool]

#model_b = Tfidf(docs_b, doc_ids_b)
#topic_model_b = TopicModelSVD(model_b.model, model_b.cv_tfidf)

docs_a = [p['clean_doc'] for p in casting_pool]
doc_ids_a = [p['_id'] for p in casting_pool]

model_a = Tfidf(docs_a, doc_ids_a)
topic_model = TopicModelSVD(model_a.model, model_a.cv_tfidf)

In [5]:
## Topic Model Roles A, Roles B

a_roles_topic_mdl = get_SVD_model(a_roles)
#b_roles_topic_mdl = get_SVD_model(b_roles)

In [6]:
## Cluster
cpool_a_kmeans = KMeans(n_clusters=10)
#cpool_b_kmeans = KMeans(n_clusters=10)

cpool_a_kmeans.fit(model_a.model)
#cpool_b_kmeans.fit(model_b.model)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [ ]:
## Match

In [7]:
a_roles_tfidf, a_roles_topics = get_SVD_model(a_roles)

In [8]:
a_roles_tfidf.model

,aa,abbreviated,abc,ability,ablation,able,aboard,abort,aborted,abraham,...,القمر,انشقاق,に野口聡一,ぴあ映画生活,オルドリンが出演,バズ,宇宙兄弟,小栗旬,岡田将生主演,玉兔
5c75dbadfc0c4b2a44860227,0.004781,0.009096,0.000000,0.003032,0.000000,0.003870,0.007741,0.015159,0.004781,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5c75dbadfc0c4b2a44860228,0.005716,0.000000,0.003625,0.000000,0.000000,0.002314,0.004627,0.000000,0.000000,0.076123,...,0.000000,0.000000,0.003625,0.003625,0.003625,0.003625,0.003625,0.003625,0.003625,0.000000
5c75dbadfc0c4b2a44860229,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5c75dbadfc0c4b2a4486022a,0.000000,0.000000,0.000000,0.000000,0.003222,0.002056,0.002056,0.000000,0.002540,0.000000,...,0.003222,0.003222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003222


In [9]:
for role in a_roles:
    print (role['name'] + '\'s id is: ' + str(role['_id']))

Neil Armstrong's id is: 5c75dbadfc0c4b2a44860227
Buzz Aldrin's id is: 5c75dbadfc0c4b2a44860228
Michael Collins \(astronaut\)'s id is: 5c75dbadfc0c4b2a44860229
Moon's id is: 5c75dbadfc0c4b2a4486022a


## Get a cluster, but also translates clean text to proper vector-space

In [17]:
#Transform Roles A & B accoring to models of Roles:
import pandas as pd

docs_a_r = [p['clean_doc'] for p in a_roles]
doc_ids_a_r = [p['_id'] for p in a_roles]

a_roles_transformed = model_a.cv_tfidf.transform(docs_a_r).toarray()


#import numpy as np
#for role in a_roles:
#    cpool_a_kmeans.predict(np.array(a_roles_transformed.loc[role['_id']]).reshape(1,-1))
cpool_a_kmeans.predict(np.array(a_roles_transformed[0]).reshape(1,-1))

array([8], dtype=int32)

In [66]:
#Greg Gumbel was placed in group 9. Choose a random candidate to play him in the movie. 
candidates = []
for i in zip(cpool_a_kmeans.labels_, a_cstng_pool):
    if i[0] == 9:
        candidates.append((i[1]['name']))

print (f'{np.random.choice(candidates)} chosen to play Greg Gumbel')

Michael Lerner (actor) chosen to play Greg Gumbel


# Given a person and cluster, gives the best match

In [18]:

min_ = -np.inf
name = ''
person = None
for i in zip(cpool_a_kmeans.labels_, casting_pool):
    
    #Serach the correct cluster
    if i[0] == 8:
        
        #a_roles_transformed is a role
        #model_a is the casting pool vector tfdidf
        dist = cosine_distance(np.array(a_roles_transformed[0]), model_a.model.loc[i[1]['_id']])
        
        if dist > min_:
            min_ = dist
            name = i[1]['name']
            person = i[1]
            print(name + ' ' + str(dist))
            
print (name)

Tom Cruise 0.024861821529126554
Robert De Niro 0.025993339460747677
Anthony Hopkins 0.02740181802737521
Tom Hanks 0.04409065718578947
Robert Duvall 0.04578903560948069
Judy Garland 0.09485885507881404
Judy Garland


In [ ]:
Neil Armstrong & Buzz Aldrin - Judy Garland, Michael Collins - Merlin the Magic Mouse, Sam Rockwell - Moon

In [20]:
topic_model.display_topics(10)


Topic  0
film, award, best, actress, actor, nominated, supporting, role, critics, edit

Topic  1
album, music, band, released, song, single, tour, billboard, songs, parser

Topic  2
streep, parser, mw, output, quotebox, cs, bugs, film, hoffman, looney

Topic  3
bugs, looney, bunny, tunes, daffy, duck, porky, sylvester, tweety, cartoon

Topic  4
cs, parser, mw, output, lock, moore, svg, williams, em, winslet


In [87]:
from bson.objectid import ObjectId
greg_gumbel = ObjectId('5c6f04ccfc0c4b94ecceef0e')
a_roles_tfidf.model.loc[greg_gumbel]

pandas.core.series.Series

In [93]:
def cosine_distance(u, v):
    """
    Returns the cosine of the angle between vectors v and u. This is equal to
    u.v / |u||v|.
    """
    return numpy.dot(u, v) / (math.sqrt(numpy.dot(u, u)) * math.sqrt(numpy.dot(v, v)))

In [13]:
from nlp.models import Tfidf, TopicModelSVD

def get_SVD_model(people):
    print(people)
    
    docs = [p['clean_doc'] for p in people]
    doc_ids = [p['_id'] for p in people]
    
    model = Tfidf(docs, doc_ids)
    topic_model = TopicModelSVD(model.model, model.cv_tfidf)
    
    return (model, topic_model)

In [3]:
cast = cpool.get_casting_pool()
docs = [p['stemmed_wiki_doc'] for p in cast]
doc_ids = [p['_id'] for p in cast]



model = Tfidf(docs, doc_ids)
topic_model = TopicModelSVD(model.model, model.cv_tfidf)

In [5]:
topic_model = TopicModelSVD(model.model, model.cv_tfidf)

In [58]:
from sklearn.cluster import KMeans
means = KMeans(n_clusters=15, random_state=51)

In [ ]:
means = KMeans(n_clusters=15, random_state=51)

In [59]:
means.fit(model.model)

MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10, n_clusters=20,
        n_init=3, random_state=51, reassignment_ratio=0.01, tol=0.0,
        verbose=0)

In [60]:
means.inertia_

738.669601596358

In [39]:
np.array(model.model).shape

(1060, 80806)

In [67]:
means.predict()

ValueError: Expected 2D array, got 1D array instead:
array=[0. 0. 0. ... 0. 0. 0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [111]:
min_ = -np.inf
name = ''
person = None
for i in zip(cpool_a_kmeans.labels_, a_cstng_pool):
    
    #Serach the correct cluster
    #if i[0] == 0:
        
    #a_roles_transformed is a role
    #model_a is the casting pool vector tfdidf
    dist = cosine_distance(np.array(a_roles_transformed[0]), model_a.model.loc[i[1]['_id']])

    if dist > min_:
        min_ = dist
        name = i[1]['name']
        person = i[1]
        print(name + ' ' + str(dist))

print (name)

Peter O'Toole 0.12631355092771873
Richard Burton 0.1330834600937386
Laurence Olivier 0.1361747203832702
Rod Steiger 0.14307874182390548
James Earl Jones 0.14677735250988222
Woody Allen 0.14899710934935617
Jeff Bridges 0.17907671371299091
Robin Williams 0.1797301772873159
Will Smith 0.24539435597755532
Samuel L. Jackson 0.4274059203214055
Samuel L. Jackson


In [142]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts
import multiprocessing
cores = multiprocessing.cpu_count()
import gensim.models.doc2vec

assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"



In [130]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
documents[0]

TaggedDocument(words=['human', 'interface', 'computer'], tags=[0])

In [139]:
docs = [TaggedDocument(person['stemmed_wiki_doc'].split(), [person['_id']]) for person in a_cstng_pool]

In [140]:
model = Doc2Vec(docs, vector_size=5, window=2, min_count=2, workers=4)